In [0]:
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "<client-id>",
    "fs.azure.account.oauth2.client.secret": "<client-secret>",
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<tenant-id>/oauth2/token"
}

# Replace <storage-account-name> and <container-name> with your details
dbutils.fs.mount(
    source = "abfss://<container-name>@<storage-account-name>.dfs.core.windows.net/",
    mount_point = "/mnt/datalake",
    extra_configs = configs
)


In [0]:
dbutils.fs.ls("/mnt/datalake")

[FileInfo(path='dbfs:/mnt/datalake/raw_data/', name='raw_data/', size=0, modificationTime=1733606709000),
 FileInfo(path='dbfs:/mnt/datalake/transformed_data/', name='transformed_data/', size=0, modificationTime=1733606724000)]

In [0]:
athelete_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/datalake/raw_data/athelete.csv")
coaches_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/datalake/raw_data/coaches.csv")
gender_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/datalake/raw_data/gender.csv")
medals_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/datalake/raw_data/medals.csv")
teams_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/datalake/raw_data/teams.csv")

In [0]:
medals_df.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Team/NOC: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
import pyspark.sql.functions as F


In [0]:
# Countries by number of gold medals
medals_df = medals_df.withColumnRenamed("Team/NOC","Team")
medals_df.orderBy("Gold", ascending=False).select("Team","Gold").show()

+--------------------+----+
|                Team|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|                Cuba|   7|
|         New Zealand|   7|
|              Brazil|   7|
|              Canada|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|               Kenya|   4|
|              Poland|   4|
|      Czech Republic|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [0]:
athelete_df.show()

+--------------------+--------------------+-------------------+
|                Name|                 NOC|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
from pyspark.sql.window import Window
# Calculate most popular entries by each country
# Step 1: Count occurrences of each discipline by country
discipline_counts = (
    athelete_df.groupBy("NOC", "Discipline")
      .agg(F.count("*").alias("count"))
)

# Step 2: Use a window function to find the discipline with the maximum count per country
window_spec = Window.partitionBy("NOC").orderBy(F.col("count").desc())

most_popular = (
    discipline_counts.withColumn("rank", F.row_number().over(window_spec))
                     .filter(F.col("rank") == 1)  # Select the top discipline for each country
                     .drop("rank")
)

# Show Results
most_popular.orderBy("Count", ascending=False).show(truncate=False)

+--------------------------+-----------------+-----+
|NOC                       |Discipline       |count|
+--------------------------+-----------------+-----+
|United States of America  |Athletics        |144  |
|Germany                   |Athletics        |95   |
|Great Britain             |Athletics        |75   |
|Italy                     |Athletics        |73   |
|Japan                     |Athletics        |70   |
|France                    |Athletics        |66   |
|Australia                 |Athletics        |65   |
|Poland                    |Athletics        |64   |
|Spain                     |Athletics        |58   |
|Canada                    |Athletics        |56   |
|Jamaica                   |Athletics        |54   |
|Brazil                    |Athletics        |53   |
|People's Republic of China|Athletics        |53   |
|Ukraine                   |Athletics        |48   |
|Netherlands               |Athletics        |44   |
|Ethiopia                  |Athletics        |